In [7]:
import sys
import json
import base64
import os
# import pandas as pd

from urllib.request import urlopen
from urllib.request import Request
from urllib.error import URLError
from urllib.parse import urlencode
from urllib.parse import quote_plus

In [5]:
def print_json(data):
    print(json.dumps(data, sort_keys=True, indent=4, separators=(', ', ': '), ensure_ascii=False))

In [16]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

TOKEN_URL = 'https://aip.baidubce.com/oauth/2.0/token'
API_KEY = '9NVR7wXLzVQEzulHiDiUO1WF'
SECRET_KEY = 'LydgIkbuw9DcT81ZEnDlr4hWrhpeh29v'
OCR_URL = "https://aip.baidubce.com/rest/2.0/ocr/v1/idcard"

def fetch_token():
    params = {'grant_type': 'client_credentials',
              'client_id': API_KEY,
              'client_secret': SECRET_KEY}
    post_data = urlencode(params)
    post_data = post_data.encode('utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req, timeout=5)
        result_str = f.read()
    except URLError as err:
        print(err)
    result_str = result_str.decode()
    result = json.loads(result_str)

    if ('access_token' in result.keys() and 'scope' in result.keys()):
        if not 'brain_all_scope' in result['scope'].split(' '):
            print ('please ensure has check the  ability')
            exit()
        return result['access_token']
    else:
        print('please overwrite the correct API_KEY and SECRET_KEY')
        exit()
        
access_token = fetch_token()

In [17]:
import difflib
import Levenshtein

def get_equal_rate_1(str1, str2):
   return difflib.SequenceMatcher(None, str1, str2).quick_ratio()

def get_equal_rate_2(str1, str2):
   return Levenshtein.ratio(str1, str2)

if __name__ == '__main__':
    a = '123456a'
    b = '234567b'
    
    print(get_equal_rate_1(a,b))
    print(get_equal_rate_2(a,b))

ModuleNotFoundError: No module named 'Levenshtein'

In [18]:
df_provinces = pd.read_csv('./Postprocessing/data/provinces.csv')
df_district = pd.read_csv('./Postprocessing/data/district.csv')

areacode_unit_address = json.load(open('Postprocessing/data/areacode_unit_address.json', "r", encoding="utf-8"))

NameError: name 'pd' is not defined

# 身份证

# 户口本

- words_result_num：识别结果数，表示words_result的元素个数
- words_result：识别结果
  - BirthAddress：出生地
  - Birthday：生日
  - CardNo：身份证号
  - Name：姓名
  - Nation：民族
  - Relationship：与户主关系
  - Sex：性别

In [21]:
# encoding:utf-8
# from Postprocessing.fix_birth import fix_birthday
# from Postprocessing.fix_idnum import fix_idnum
# from Postprocessing.fix_birthplace import fix_birthplace

import requests
import base64

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/household_register"
# 二进制方式打开图片文件
f = open(r'C:\Users\ZHOU-JC\Desktop\2.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
result = response.json()['words_result']

d = dict(zip(['BirthAddress', 'Birthday', 'CardNo', 'Name', 'Nation', 'Relationship', 'Sex'], ['出生地', '生日', '身份证号', '姓名', '民族', '与户主关系', '性别']))
for key in d:
    if key in result.keys():
        result[d[key]] = result.pop(key)['words']
result
# temp1 = pd.DataFrame(result, index=[1])

# # 修正出生日期
# if '生日' in result and '身份证号' in result:
#     result['生日'] = fix_birthday(result['生日'], result['身份证号'])
# # 修正身份证号
# if '生日' in result and '身份证号' in result and '出生地' in result:
#     result['身份证号'] = fix_idnum(result['身份证号'], result['生日'], result['出生地'])
# # 修正出生地 
#     result['出生地'] = fix_birthplace(result['出生地'], result['身份证号'], df_district, areacode_unit_address)
# print('\n\n')
# temp2 = pd.DataFrame(result, index=[2])

# temp = temp1.append(temp2)
# temp.T

{'出生地': '四川省南充市顺庆区',
 '生日': '990年12月14日',
 '身份证号': '511302199012143211',
 '姓名': '郭骥',
 '民族': '汉族',
 '与户主关系': '子',
 '性别': '男'}

# 毕业证

In [36]:
import requests
import json
import cv2
import base64

def cv2_to_base64(image):
    data = cv2.imencode(r'/home/zhoujx/4.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

headers = {"Content-type": "application/json"}
url = "http://192.168.63.8:18889/predict/chinese_ocr_db_crnn_mobile"

img_path = '/home/zhoujx/OCR/证件/营业执照/2447_784081ed5021d6d673f980af3269d727.jpg'
img = cv2.imread(img_path)
img_base64 = cv2_to_base64(img)
# print(img_base64)

data = {'images': [img_base64]}
r = requests.post(url=url, headers=headers, data=json.dumps(data))
# print(r.json())
result = r.json()["results"]
print(result)
result_info = result[0]['data']
for re in result_info:
    print(re['text'])

[{'data': [{'confidence': 0.5824835896492004, 'text': '一美', 'text_box_position': [[595, 40], [690, 29], [694, 60], [600, 70]]}, {'confidence': 0.9727228879928589, 'text': '营业执照', 'text_box_position': [[238, 273], [722, 270], [722, 386], [238, 389]]}, {'confidence': 0.6467512845993042, 'text': '人园外卖', 'text_box_position': [[118, 364], [195, 326], [212, 357], [135, 394]]}, {'confidence': 0.962008535861969, 'text': '注册号530902600200836', 'text_box_position': [[457, 418], [707, 418], [707, 446], [457, 446]]}, {'confidence': 0.8714941740036011, 'text': '称', 'text_box_position': [[275, 482], [310, 482], [310, 513], [275, 513]]}, {'confidence': 0.8786761164665222, 'text': '临翔区肖芳蛋糕店', 'text_box_position': [[328, 485], [501, 486], [501, 512], [328, 510]]}, {'confidence': 0.7768590450286865, 'text': '类', 'text_box_position': [[191, 538], [224, 538], [224, 572], [191, 572]]}, {'confidence': 0.9558396935462952, 'text': '型', 'text_box_position': [[274, 541], [311, 541], [311, 570], [274, 570]]}, {'c

# 营业执照识别

In [5]:
# encoding:utf-8

import requests
import base64

'''
营业执照识别
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/business_license"
# 二进制方式打开图片文件
f = open(r'D:\Projects\sendi\OCR\Identify_Cards\data\2702_69a1c31c27961bdd67d34208bee216b3.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print_json(response.json())

{
    "direction": 1, 
    "log_id": 1303945317103697921, 
    "words_result": {
        "单位名称": {
            "location": {
                "height": 13, 
                "left": 193, 
                "top": 296, 
                "width": 166
            }, 
            "words": "宁蒗县永宁乡泸沽湖馋嘴馆"
        }, 
        "地址": {
            "location": {
                "height": 12, 
                "left": 195, 
                "top": 367, 
                "width": 280
            }, 
            "words": "云南省丽江市宁蔻彝族自治县永宁乡大落水村"
        }, 
        "成立日期": {
            "location": {
                "height": 13, 
                "left": 195, 
                "top": 449, 
                "width": 98
            }, 
            "words": "2017年03月12日"
        }, 
        "有效期": {
            "location": {
                "height": 0, 
                "left": -1, 
                "top": -1, 
                "width": 0
            }, 
            "words": "无"
        }, 
        "法人": {
        

# 出租车

In [7]:
# encoding:utf-8

import requests
import base64

'''
出租车票识别
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/taxi_receipt"
# 二进制方式打开图片文件
f = open('C:/Users/ZHOU-JC/Desktop/938.png', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())

{'log_id': 3663826279698977993, 'words_result_num': 9, 'words_result': {'Fare': '¥20.00元', 'InvoiceNum': '12296920', 'InvoiceCode': '131001861704', 'FuelOilSurcharge': '¥0.00', 'TotalFare': '¥20.00', 'TaxiNum': '131001861704', 'Time': '18:51-19:41', 'Date': '2018-11-15', 'CallServiceSurcharge': '¥0.00'}}


# 公司API

In [3]:
import requests
import json
import cv2
import base64

def cv2_to_base64(image):
    data = cv2.imencode(r'.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

headers = {"Content-type": "application/json"}
url = "http://192.168.63.8:18889/predict/chinese_ocr_db_crnn_mobile"

img_path = r'C:\Users\ZHOU-JC\Desktop\733.png'
img = cv2.imread(img_path)
img_base64 = cv2_to_base64(img)
# print(img_base64)

data = {'images': [img_base64]}
r = requests.post(url=url, headers=headers, data=json.dumps(data))
# print(r.json())
result = r.json()["results"]
print(result)
result_info = result[0]['data']
for re in result_info:
    print(re['text'])

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


[{'data': [{'confidence': 0.8599449396133423, 'text': '广东东党市', 'text_box_position': [[301, 45], [689, 23], [694, 88], [306, 109]]}, {'confidence': 0.9478442072868347, 'text': '发票代码', 'text_box_position': [[43, 286], [363, 300], [358, 376], [37, 362]]}, {'confidence': 0.9974994659423828, 'text': '144191770311', 'text_box_position': [[360, 295], [963, 295], [963, 376], [360, 376]]}, {'confidence': 0.9689872860908508, 'text': '发票号码', 'text_box_position': [[40, 376], [350, 395], [344, 469], [35, 450]]}, {'confidence': 0.9941022396087646, 'text': '00950645', 'text_box_position': [[376, 391], [888, 391], [888, 472], [376, 472]]}, {'confidence': 0.944604218006134, 'text': '投诉电话0769-22301160', 'text_box_position': [[32, 486], [915, 493], [915, 574], [32, 567]]}, {'confidence': 0.6923086643218994, 'text': '单位名称东党市期选集饼有', 'text_box_position': [[29, 570], [972, 589], [969, 698], [26, 679]]}, {'confidence': 0.9864304065704346, 'text': '机打发票', 'text_box_position': [[312, 834], [630, 839], [630, 913

# 通用文字识别（标准含位置版）

In [77]:
# encoding:utf-8

import requests
import base64

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general"
# 二进制方式打开图片文件
f = open('/home/zhoujx/OCR/证件/营业执照/2956_2623a3d4c9e3c7c80eb43fe0f06ea275.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())
print_json(response.json())

{'words_result': [{'words': '★', 'location': {'top': 397, 'left': 2108, 'width': 45, 'height': 39}}, {'words': '★', 'location': {'top': 397, 'left': 2250, 'width': 47, 'height': 43}}, {'words': '统社会信用代码营业执照', 'location': {'top': 769, 'left': 610, 'width': 2770, 'height': 253}}, {'words': '扫描二维码登录', 'location': {'top': 848, 'left': 3386, 'width': 234, 'height': 39}}, {'words': '国家企业信用', 'location': {'top': 893, 'left': 3421, 'width': 202, 'height': 43}}, {'words': '信息公示系统', 'location': {'top': 943, 'left': 3390, 'width': 200, 'height': 37}}, {'words': '了解更多登记、', 'location': {'top': 988, 'left': 3394, 'width': 212, 'height': 41}}, {'words': '(副本)副本编号:', 'location': {'top': 1037, 'left': 2061, 'width': 600, 'height': 102}}, {'words': '备案,许可、监', 'location': {'top': 1035, 'left': 3398, 'width': 226, 'height': 37}}, {'words': '管信息', 'location': {'top': 1080, 'left': 3392, 'width': 112, 'height': 51}}, {'words': '名', 'location': {'top': 1317, 'left': 641, 'width': 86, 'height': 82}}, {'words'

# 通用文字识别

In [12]:
# encoding:utf-8

import requests
import base64

'''
通用文字识别（高精度版）
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate_basic"
# 二进制方式打开图片文件
f = open(r'C:\Users\ZHOU-JC\Desktop\1\682.png', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())

{'log_id': 3101690453819464522, 'words_result_num': 28, 'words_result': [{'words': '本发票开具合计金额超过万元无效'}, {'words': '广东省广市出租汽车统车票'}, {'words': ' GD. GUANGZHOU-TAXI RECEIPT'}, {'words': '发家票监联'}, {'words': '144011770326'}, {'words': '00987909'}, {'words': '监督电话32034257'}, {'words': '黄企运输有限'}, {'words': '9140110'}, {'words': '发票专用章'}, {'words': '此发票手写无效'}, {'words': '电话2257'}, {'words': '车号粤4T6'}, {'words': '证号11111'}, {'words': '日期1117日'}, {'words': '上车'}, {'words': '2116'}, {'words': '下车'}, {'words': '2143'}, {'words': '单价'}, {'words': '6元'}, {'words': '里程'}, {'words': '候时'}, {'words': '金额'}, {'words': '元'}, {'words': '卡号'}, {'words': '准黄国税自印[2017]020号(101017)人印AJ0706'}, {'words': '金额超过万元无效'}]}


# 爬虫

In [1]:
from PIL import Image
import zxing  # 导入解析包


def ocr_qrcode_zxing(filename):
    img = Image.open(filename)
    zx = zxing.BarCodeReader()  # 调用zxing二维码读取包
    data = ''
    zxdata = zx.decode(filename)  # 图片解码
    return zxdata.parsed  # 返回记录的内容

ModuleNotFoundError: No module named 'zxing'